# Assignment 1

#### Student ID: *Double click here to fill the Student ID*

#### Name: *Double click here to fill the name*

In [ ]:
%pip install --pre pycaret[full] -qq
%pip install cleanlab -qq
%pip install snorkel -qq

## Q1: Exploring a Low-Code ML Framework with Melbourne Housing dataset

After completing the California census data project, we will shift our focus to a different housing dataset.

The dataset is a snapshot of a [dataset](https://www.kaggle.com/datasets/anthonypino/melbourne-housing-market) created by Tony Pino. This dataset is a compilation of housing market data for Melbourne, which was sourced from weekly public listings on [Domain.com.au](https://www.domain.com.au/). It includes various details such as the address, type of real estate, suburb, selling method, number of rooms, price, real estate agent, date of sale, and the distance from the central business district, among others. **Ensure to set the random seed to 2024 for reproducibility.**

To begin, execute the following code snippet for data preparation:


In [ ]:
import numpy as np
import pandas as pd
from pycaret.regression import *

# Prepare data
df = pd.read_csv("melb_data.csv")
df = df.drop(['Date', 'Address'], axis=1)
df = df.copy()
df

**(a) Data Splitting**: Using the `train_test_split()` function, divide the dataset into two parts: a training set and a test set. Allocate 5% of the data to the test set, which will be used for testing purposes. (5%)

In [ ]:
# coding your answer here.

**(b) Dataset Preparation Using PyCaret:**
Prepare the dataset for our machine learning model using the `setup()` function from `PyCaret` with the following requirements: (10%)

- **Features**: Include all features in the `df` dataframe, except for the `Price` column, which will be used as the target.
- **Target**: Set `Price` as the target variable.
- **Validation Set**: Split 20% of the data into a validation set to evaluate the model.
- **Missing Values**:
  - For numerical variables, replace missing values with the median value of the respective columns. Assume that columns of data type `object` are categorical variables, and all others are numerical.
  - For categorical variables, replace missing values with the most frequent value in each column.
- **Encoding**:
  - Apply one-hot encoding to categorical variables that have fewer than 10 categories.
  - Use target encoding for categorical columns with 10 or more categories.
- **Standardization**: All features should be standardized. Note that PyCaret automatically scales all features by default.

**Hint:** Refer to the PyCaret documentation [here](https://pycaret.readthedocs.io/en/latest/api/regression.html) for additional details on setup parameters. Be aware of how PyCaret handles feature scaling by default as discussed [here](https://github.com/pycaret/pycaret/issues/3076).

In [ ]:
# coding your answer here.

After setting up your dataset with `PyCaret`, you can review the configuration and steps of the created machine learning pipeline by executing the following code snippet. This will help you verify that all preprocessing steps are correctly applied as per the requirements:

In [ ]:
# Get the PyCaret configuration
config = get_config('pipeline')
config

**(c) Verify Transformed Data:**
Use the `get_config()` function in `PyCaret` to inspect the transformed dataset. After executing this function, review the output to ensure the number of samples and the number of features match your expectations based on the data preparation steps previously outlined. (10%)

Provide a brief explanation of why the numbers match or differ from your expectations.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

**(d) Model Comparison and Evaluation:** (10%)

Utilize the `compare_models()` function in `PyCaret` to conduct a 3-fold cross-validation analysis. Compare the performance of the following machine learning models:

- Linear Regression
- Lasso Regression
- Ridge Regression
- Support Vector Regression (SVR)
- K-Nearest Neighbor (KNN) Regressor
- Random Forest Regressor
- XGBoost
- LightGBM
- CatBoost

List the top five models according to their performance on the Mean Absolute Percentage Error (MAPE) measure.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

**(e) Final Model Selection and Evaluation**: Using the `create_model()` function in `PyCaret`, select the best-performing model from part (d) based on the MAPE metric. Evaluate this model on the test set that was reserved in part (a). Report the MAPE on the test set to assess the model’s performance. (5%)

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

## Q2: Handling an Unlabeled Dataset in a Startup Environment

In this exercise, you'll hone your data preparation skills, a crucial element in many real-world projects. You are provided with an unlabeled dataset `data.csv`, which consists of two features and encompasses 300 sample points. Each sample point is associated with one of four distinct classes. Additionally, you have `crowdsourcing.csv` containing labels from 50 different workers for these sample points.

**While this is a practice project, and you have access to the actual ground truth labels in `labels.csv`, these labels are restricted to measuring the accuracy of your models. Under no circumstances should the ground truth labels be used in other parts of the training process.** Ensure to set the random seed to 2024 for all operations requiring randomness to maintain consistency in your results.

Firstly, read the dataset and divide it into training and testing sets using the following code snippet:

In [ ]:
X = pd.read_csv('data.csv', names=[0,1])
true_labels = np.loadtxt("labels.csv", delimiter=',')
multiannotator_labels = pd.read_csv('crowdsourcing.csv')

In [ ]:
X_train, X_test, multiannotator_labels, ano_unseen, Y_train, Y_test  = train_test_split(X, multiannotator_labels, true_labels, test_size=0.1, random_state=2024)

**(a) Label Aggregation Using Snorkel:** (10%)

In this task, we'll explore the use of `Snorkel` to manage and refine crowdsourced labels. Each crowd worker will be treated as an individual labeling function since each one labels different subsets of the dataset and might introduce errors or conflicts.

**Steps:**

1. **Convert Crowdsourcing Data**: Transform the crowdsourcing data frame into a label matrix where each row corresponds to a sample and each column to a worker's label for that sample. This matrix will be used to train the `Snorkel` model.

2. **Train the Label Model**: Use the `Snorkel` `LabelModel` to train on the label matrix you created. Apply the following hyperparameters:
   ```python
   fit(L_train, n_epochs=500, seed=2024, log_freq=20, l2=0.1, lr=0.001, lr_scheduler="linear", optimizer="adam")
   ```

3. **Generate Predictions and Evaluate Accuracy**: Once trained, use the `LabelModel` to generate predictions for each training sample. Then, calculate the accuracy of these predictions by comparing them against the ground truth labels from `labels.csv`. Remember, the ground truth is only used for evaluating accuracy, not for training the Label Model.

**Hints:**
- You can check the Snorkel documentation [here](https://snorkel.readthedocs.io/en/master/packages/_autosummary/labeling/snorkel.labeling.model.label_model.LabelModel.html) for more details on the `LabelModel` parameters and methods.

In [ ]:
# coding your answer here.

**(b) Refining Labels with Cleanlab:** (10%)

In this second trial, we'll leverage the `cleanlab` library to further analyze and refine the labels obtained from crowdsourcing. The process will involve deriving consensus labels from the crowdsourced data, using majority voting and model predictions.

**Steps:**

1. **Majority Vote Labels**:
   - Utilize the `get_majority_vote_label()` function to aggregate the labels from different workers for each sample into a single majority vote label.
   
2. **Model Predictions**:
   - Apply a 5-fold cross-validation on the training dataset labeled with the majority vote results. Use the model below to compute out-of-sample predicted probabilities for each sample.
   ```python
    LogisticRegression(solver='liblinear', penalty='l1', C=0.1, random_state=2024)
   ```

3. **Consensus Labels**:
   - Employ the `get_label_quality_multiannotator()` function from `cleanlab` to combine the out-of-sample predicted probabilities with the crowdsourced labels. This function will help generate consensus labels, which aim to be more accurate by considering both worker disagreement and model confidence.

4. **Evaluate Accuracy**:
   - Calculate the accuracy of both the majority vote labels and the consensus labels by comparing them against the ground truth labels in `labels.csv`. As with previous tasks, remember that the ground truth is strictly for evaluation purposes.

**Hints:**
- Explore the Cleanlab documentation [here](https://github.com/cleanlab/cleanlab) for additional insights into functions like `get_majority_vote_label()` and `get_label_quality_multiannotator()`.

In [ ]:
# coding your answer here.

**(c) Evaluate Random Forest Classifiers:** (10%)

Since we don't have these crowdsourcing labels for the test set or new incoming data points, we can't use the `LabelModel` or obtain consensus label at inference time. In order to run inference on new incoming data points, we need to train a discriminative model! Train a model using the following setup:

```python
RandomForestClassifier(random_state=2024, n_estimators=10)
```
**Steps:**
- Train a classifier for each type of label.
    - **Snorkel Labels**: Generated in part (a).
    - **Majority-Vote Labels**: Obtained in part (b) from `get_majority_vote_label()`.
    - **Consensus Labels**: Derived in part (b) from `get_label_quality_multiannotator()`.
- Evaluate and report the classification accuracy on the test set.
- Determine which classifier performs best based on the accuracy scores.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

**(d) Enhanced Random Forest Training:** (10%)

Train two additional Random Forest classifiers with the specifications:
```python
RandomForestClassifier(random_state=2024, n_estimators=10)
```

**Steps:**
- Fit each classifier using the respective sample weights for the labels.
    - **Snorkel Labels**: Use `inv_entropy()` to calculate weights based on prediction probabilities from the `Snorkel` `LabelModel`.
    - **Consensus Labels**: Use `consensus_quality_score` from `Cleanlab` for weights.
- Evaluate and report the classification accuracy on the test set.
- Compare the results to those from (c) to assess performance improvement.

Hint: Refer to [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.fit) for the `sample_weight` paprameter.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

## Q3: Analyze StackOverflow dataset using SQL

Kaggle has a rich number of [BigQuery](https://www.kaggle.com/datasets?fileType=bigQuery) and [SQLite](https://www.kaggle.com/datasets?fileType=sqlite) datasets that you can practice your SQL skill.

In this question, we are going to examine the StackOverflow dataset. [Stack Overflow](https://stackoverflow.com/) is a popular question and answer site for technical questions.

Hint: It is recommended to answer this question in Kaggle, where you can access the dataset directly.

Firstly, if you are using colab, use the following code snippet to setup the client. For more detail, please refer to our lab.

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import sys

# Upload JSON file here
from google.colab import files
uploaded = files.upload()

path_of_json = 'nomadic-botany-435608-b6-e1a2d53f264a.json'  #@param {type: "string"}

# TODO(developer): Set path_of_json to the path to the service account key file.

credentials = service_account.Credentials.from_service_account_file(
    path_of_json
)

In [ ]:
if "google.colab" in sys.modules:
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
else:
    # Below and create a "Client" object if you are using Kaggle
    client = bigquery.Client()

**(a) Initial Exploration of the Stack Overflow Dataset:** (10%)

Begin your analysis by familiarizing yourself with the structure and contents of the Stack Overflow dataset:

1. **List All Tables**: Print the names of all tables available in the dataset.
2. **View Table Schemas**: Display the schema for the `posts_questions` and `posts_answers` tables.
3. **Preview Data**: Retrieve and preview the first ten rows from each of the tables mentioned above.

**Hint:** Use the following code snippet to reference the dataset in your client setup:
```python
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
```

In [ ]:
# coding your answer here.

**(b) Daily Post Counts for 2020:** (10%)

Write an SQL query to count the number of posts created each day in the `posts_questions` table for the year 2020:

- **Filter by Year**: Use `WHERE` and `EXTRACT()` to select posts from 2020 only.
- **Date Column**: Format the `creation_date` in DATE format (e.g. 2021-01-01) and rename it to "created_day". (Remember the SELECT... from clause, DATE() function and `AS` command).
- **Count Posts**: Count the daily posts and label this column as "number_of_posts". (Remember the `COUNT()` function).
- **Group and Sort**: Group the results by "created_day" using `GROUP BY` and order them by "created_day" in ascending order using `ORDER BY`.

Your query should return a dataframe with 366 rows and two columns, showing the date and corresponding post count.

<center>

| created_day | number_of_posts |
|:-----------:|:---------------:|
|  2020-01-01 |       2390      |
|  2020-01-02 |       4601      |
|  2020-01-03 |       4816      |
|     ...     |       ....   |
|  2020-12-31 |       3498      |

</center>

Generate a line plot with "created_day" on the x-axis and "number_of_posts" on the y-axis to visualize daily posting trends. What patterns do you observe from the line plot regarding post frequency throughout 2020?

In [ ]:
# coding your answer here.
query = """
     """

############   Do not modify the code below     ############
# Set up the query (cancel the query if it would use too much of
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()
############   Do not modify the code above     ############

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

**(c) Analyzing User Engagement with 'pandas' Tag**: (10%)

`posts_questions` has a column called tags which lists the topics/technologies each question is about. `posts_answers` has a column called `parent_id` which identifies the ID of the question each answer corresponds to. You can then join two tables by the `parent_id` in `posts_answers` and the `id` in `posts_questions`. `posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

Now, write a query that has a single row for each user who answered at least one question with a tag that equals to string `pandas`. Your results should have two columns (Remember the `SELET...WHERE` clause) and follow the below restrictions:

* `owner_user_id` - contains the `owner_user_id` column from the `posts_answers` table (Remember the `INNER JOIN` clause)
* `number_of_answers` - contains the number of answers the user has written to `pandas` related questions using the `tage` column (Remember the `AS`, `GROUP BY` and `COUNT` clause)
* Discard the rows whose `number_of_answers` is smaller or equal to 3 (Remember the `HAVING` clause)

<center>

| owner_user_id | number_of_answers |
|---------------|-------------------|
| 1234567       | 794               |
| 2345678       | 304               |
|     ...     |       ....   |
|  9999999 |       4      |

</center>

In the retrieve data frame, which user answers most questions in the pandas domain? Try to find the email of the user.

Hint: You can find the public user profile by appending the ID after https://stackoverflow.com/users/

In [ ]:
# coding your answer here.
query = """
        """

############   Do not modify the code below     ############
# Set up the query (cancel the query if it would use too much of
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()
############   Do not modify the code above     ############

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*